In [ ]:
from dotenv import load_dotenv,find_dotenv
from langchain_core.prompts import PromptTemplate,ChatPromptTemplate
from langchain_core.messages import SystemMessage,HumanMessage,AIMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain.prompts import (SystemMessagePromptTemplate,
                    HumanMessagePromptTemplate)

from langchain_core.tools import tool
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticToolsParser
from langchain_core.messages import ToolMessage
from langchain_core.runnables import RunnablePassthrough

In [6]:
load_dotenv(dotenv_path=find_dotenv(filename="../.env"))

True

In [7]:
llmGemini=ChatGoogleGenerativeAI(model="gemini-2.0-flash-001")
llmOpenAI=ChatOpenAI(model="gpt-4o-mini")

In [8]:
@tool
def add(a:int,b:int) -> int:
    """
        Adds a and b
    Args:
        a: first int
        b: second int
    """
    return a+b

In [10]:
@tool
def multiply(a:int,b:int) -> int:
    """
        Multiplies a and b
    Args:
        a: first int
        b: second int
    """
    return a*b

In [11]:
### tools=[add,multiply]

In [12]:
class add(BaseModel):
    """
        Add two integers togather
    """
    a: int=Field(default=...,description="First Integer")
    b: int=Field(default=...,description="Second Integer")

In [13]:
class multiply(BaseModel):
    """
        Multiplies two integers togather
    """
    a: int=Field(default=...,description="First Integer")
    b: int=Field(default=...,description="Second Integer")

In [14]:
...

Ellipsis

In [15]:
tools=[add,multiply]

In [16]:
llmWithTools=llmGemini.bind_tools(
    tools=tools
)

In [17]:
llmWithAlwaysCall=llmGemini.bind_tools(tools=tools,
                                  tool_choice="any"  # atleast call one tool, replace with multiply or add for preference
                                 )

In [18]:
llmWithMultiply=llmGemini.bind_tools(tools=tools,
                                  tool_choice="multiply"  # atleast call one tool, replace with multiply or add for preference
                                 )

In [19]:
query="What is  3*12? Also, what is 11+49?"

In [21]:
llmWithTools.invoke(input=query).tool_calls

[{'name': 'multiply',
  'args': {'a': 3.0, 'b': 12.0},
  'id': '2a86be73-d417-411b-83dd-9b00e2312784',
  'type': 'tool_call'},
 {'name': 'add',
  'args': {'a': 11.0, 'b': 49.0},
  'id': 'df41a2cc-ce86-44c1-9fd5-ed7470adcd5d',
  'type': 'tool_call'}]

In [22]:
llmWithAlwaysCall.invoke(input=query).tool_calls

[{'name': 'multiply',
  'args': {'a': 3.0, 'b': 12.0},
  'id': '13e175ff-8b18-404d-917a-eaf34d312367',
  'type': 'tool_call'},
 {'name': 'add',
  'args': {'a': 11.0, 'b': 49.0},
  'id': '09135a74-270c-45b8-928c-c7fed6a628e5',
  'type': 'tool_call'}]

In [23]:
llmWithMultiply.invoke(input=query).tool_calls

[{'name': 'multiply',
  'args': {'a': 3.0, 'b': 12.0},
  'id': 'f6713886-d16b-43b6-97e7-7b2a408f0f2c',
  'type': 'tool_call'}]

In [24]:
chain=llmWithAlwaysCall|PydanticToolsParser(tools=[multiply,add])

In [25]:
chain.invoke(input=query)

[multiply(a=3, b=12), add(a=11, b=49)]

<h3>Streaming</h3>

In [26]:
async for chunk in llmWithTools.astream(input=query):
    print(chunk.tool_call_chunks)

[{'name': 'multiply', 'args': '{"a": 3.0, "b": 12.0}', 'id': 'e3612043-6183-49d4-85e2-430896168033', 'index': None, 'type': 'tool_call_chunk'}, {'name': 'add', 'args': '{"a": 11.0, "b": 49.0}', 'id': 'a7c0e236-7457-474e-bf00-20a2ac2732a9', 'index': None, 'type': 'tool_call_chunk'}]


In [30]:
async for chunk in llmWithTools.astream(input=query):
    print(chunk.tool_calls)

[{'name': 'multiply', 'args': {'a': 3.0, 'b': 12.0}, 'id': '475ee482-75c3-4214-8af4-423123109dea', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11.0, 'b': 49.0}, 'id': '844b2c73-d86c-4def-89e6-b5efb5c33ff1', 'type': 'tool_call'}]


In [29]:
async for chunk in llmWithTools.astream(input=query):
    print(chunk,"\n")

content='' additional_kwargs={'function_call': {'name': 'add', 'arguments': '{"a": 11.0, "b": 49.0}'}} response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001', 'safety_ratings': []} id='run-4120e713-d08c-4aac-b21f-b2952fc09c57' tool_calls=[{'name': 'multiply', 'args': {'a': 3.0, 'b': 12.0}, 'id': 'a932662c-020a-4b61-9350-3f5f2399cdac', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11.0, 'b': 49.0}, 'id': '8ef963ca-a26e-400a-bae3-37e1f307cdb7', 'type': 'tool_call'}] usage_metadata={'input_tokens': 54, 'output_tokens': 10, 'total_tokens': 64, 'input_token_details': {'cache_read': 0}} tool_call_chunks=[{'name': 'multiply', 'args': '{"a": 3.0, "b": 12.0}', 'id': 'a932662c-020a-4b61-9350-3f5f2399cdac', 'index': None, 'type': 'tool_call_chunk'}, {'name': 'add', 'args': '{"a": 11.0, "b": 49.0}', 'id': '8ef963ca-a26e-400a-bae3-37e1f307cdb7', 'index': None, 'type': 'tool_call_chunk'}] 



In [31]:
# Accumulating Chunks
first=True
async for chunk in llmWithTools.astream(input=query):
    if first:
        gathered=chunk
        first=False
    else:
        gathered+=chunk

    print(gathered.tool_call_chunks,"\n")
print(gathered)

[{'name': 'multiply', 'args': '{"a": 3.0, "b": 12.0}', 'id': '6497e820-4eec-4f73-8185-3fe737c5e613', 'index': None, 'type': 'tool_call_chunk'}, {'name': 'add', 'args': '{"a": 11.0, "b": 49.0}', 'id': 'bfad0728-ded3-4cad-bebe-77ea5c5ddc3b', 'index': None, 'type': 'tool_call_chunk'}] 

content='' additional_kwargs={'function_call': {'name': 'add', 'arguments': '{"a": 11.0, "b": 49.0}'}} response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001', 'safety_ratings': []} id='run-99c27dbf-3f99-4fa5-b698-79689dd29a7e' tool_calls=[{'name': 'multiply', 'args': {'a': 3.0, 'b': 12.0}, 'id': '6497e820-4eec-4f73-8185-3fe737c5e613', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11.0, 'b': 49.0}, 'id': 'bfad0728-ded3-4cad-bebe-77ea5c5ddc3b', 'type': 'tool_call'}] usage_metadata={'input_tokens': 54, 'output_tokens': 10, 'total_tokens': 64, 'input_token_details': {'cache_read': 0}} tool_call_chunks=[{'name': 'multiply', 'args': '{"a": 3.0, "b": 12.0}', 'id': '6497e820-4eec-4f

In [32]:
# Accumulating Chunks
first=True
async for chunk in llmWithTools.astream(input=query):
    if first:
        gathered=chunk
        first=False
    else:
        gathered+=chunk

    print(gathered.tool_calls)

[{'name': 'multiply', 'args': {'a': 3.0, 'b': 12.0}, 'id': '037fee0d-7802-4534-90d7-2bb0399752b5', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11.0, 'b': 49.0}, 'id': '4cd3ed2e-1149-4df0-9fc0-4a9a32a79efd', 'type': 'tool_call'}]


<h3>Passing Tool Outputs to Models</h3>

In [39]:
messages=[HumanMessage(query)]

In [40]:
messages

[HumanMessage(content='What is  3*12? Also, what is 11+49?', additional_kwargs={}, response_metadata={})]

In [41]:
aiMessage=llmWithTools.invoke(input=messages)
aiMessage

AIMessage(content='', additional_kwargs={'function_call': {'name': 'add', 'arguments': '{"a": 11.0, "b": 49.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001', 'safety_ratings': []}, id='run-0bfc3495-4560-484c-b8fa-55cf0ee8869b-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3.0, 'b': 12.0}, 'id': 'fbf011b2-3253-4f49-a725-00dbc4239680', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11.0, 'b': 49.0}, 'id': '46bebb35-c8bc-4b59-a30d-ce0c4bf4fc8e', 'type': 'tool_call'}], usage_metadata={'input_tokens': 54, 'output_tokens': 10, 'total_tokens': 64, 'input_token_details': {'cache_read': 0}})

In [43]:
llmWithAlwaysCall.invoke(input=messages)

AIMessage(content='', additional_kwargs={'function_call': {'name': 'add', 'arguments': '{"a": 11.0, "b": 49.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001', 'safety_ratings': []}, id='run-3fbf18a5-498c-44c8-8a3e-b718555c71de-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3.0, 'b': 12.0}, 'id': 'e98b0b35-9e3e-4e4f-bf73-0ca779dcf366', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11.0, 'b': 49.0}, 'id': 'd024a1e1-3717-489d-8b16-b1ac02dc018a', 'type': 'tool_call'}], usage_metadata={'input_tokens': 54, 'output_tokens': 10, 'total_tokens': 64, 'input_token_details': {'cache_read': 0}})

In [44]:
aiMessage.tool_calls

[{'name': 'multiply',
  'args': {'a': 3.0, 'b': 12.0},
  'id': 'fbf011b2-3253-4f49-a725-00dbc4239680',
  'type': 'tool_call'},
 {'name': 'add',
  'args': {'a': 11.0, 'b': 49.0},
  'id': '46bebb35-c8bc-4b59-a30d-ce0c4bf4fc8e',
  'type': 'tool_call'}]

In [45]:
messages.append(aiMessage)

In [46]:
messages

[HumanMessage(content='What is  3*12? Also, what is 11+49?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'add', 'arguments': '{"a": 11.0, "b": 49.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001', 'safety_ratings': []}, id='run-0bfc3495-4560-484c-b8fa-55cf0ee8869b-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3.0, 'b': 12.0}, 'id': 'fbf011b2-3253-4f49-a725-00dbc4239680', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11.0, 'b': 49.0}, 'id': '46bebb35-c8bc-4b59-a30d-ce0c4bf4fc8e', 'type': 'tool_call'}], usage_metadata={'input_tokens': 54, 'output_tokens': 10, 'total_tokens': 64, 'input_token_details': {'cache_read': 0}})]

In [48]:
@tool
def add(a:int,b:int) -> int:
    """
        Adds a and b
    Args:
        a: first int
        b: second int
    """
    return a+b

@tool
def multiply(a:int,b:int) -> int:
    """
        Multiplies a and b
    Args:
        a: first int
        b: second int
    """
    return a*b

In [51]:
# Illustrations
ToolMessage(content=add.invoke(input= {'a': 11.0, 'b': 49.0}),tool_call_id='0ab77964-1afb-4f8c-82ea-74f55f5c0e12')

ToolMessage(content='60', tool_call_id='0ab77964-1afb-4f8c-82ea-74f55f5c0e12')

In [54]:
for toolCall in aiMessage.tool_calls:
    selectedTool={"add":add,"multiply":multiply}[toolCall['name'].lower()]
    toolOutput=selectedTool.invoke(input=toolCall['args'])
    messages.append(ToolMessage(content=toolOutput,tool_call_id=toolCall['id']))

In [55]:
messages

[HumanMessage(content='What is  3*12? Also, what is 11+49?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'add', 'arguments': '{"a": 11.0, "b": 49.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001', 'safety_ratings': []}, id='run-0bfc3495-4560-484c-b8fa-55cf0ee8869b-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3.0, 'b': 12.0}, 'id': 'fbf011b2-3253-4f49-a725-00dbc4239680', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11.0, 'b': 49.0}, 'id': '46bebb35-c8bc-4b59-a30d-ce0c4bf4fc8e', 'type': 'tool_call'}], usage_metadata={'input_tokens': 54, 'output_tokens': 10, 'total_tokens': 64, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='36', tool_call_id='fbf011b2-3253-4f49-a725-00dbc4239680'),
 ToolMessage(content='60', tool_call_id='46bebb35-c8bc-4b59-a30d-ce0c4bf4fc8e')]

In [56]:
llmWithTools.invoke(input=messages)

AIMessage(content='3 multiplied by 12 is 36, and 11 plus 49 is 60.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001', 'safety_ratings': []}, id='run-97165922-b79e-4ae8-9b1c-4eeaee1a6eb6-0', usage_metadata={'input_tokens': 70, 'output_tokens': 24, 'total_tokens': 94, 'input_token_details': {'cache_read': 0}})

In [58]:
llmWithTools.invoke(input=messages).content

'3 multiplied by 12 is 36, and 11 plus 49 is 60.'

In [59]:
len(messages)

4

<h3>Few Shot Prompting</h3>

In [60]:
examples=[
    HumanMessage(
        content="What's the product of 317253 and 128472 plus four",
        name="exampleUser"
    ),
    AIMessage(
        content="",
        name="exampleAssistant",
        tool_calls=[
            {"name":"multiply","args":{"x":317253,"y":128472},"id":"1"}
        ]
    ),
    ToolMessage(content="16505054784",tool_call_id="1"),
    AIMessage(
        content="",
        name="exampleAssistant",
        tool_calls=[
            {"name":"add","args":{"x":16505054784,"y":4},"id":"2"}
        ]
    ),
    ToolMessage(content="16505054784",tool_call_id="2"),
    AIMessage(
        content="The product of 317253 and 128472 plus four is 16505054784",
        name="exampleAssistant"
    ),
]

In [61]:
fewShotPromptTemplate=ChatPromptTemplate.from_messages(
    [
        ("system",""" 
                You are bad at math but are an expert at using a calculator. 
                Use past tool usage as an example of how to correctly use the tools.
        """),
        *examples,
        ("human","{query}")
    ]
)

In [62]:
chain=RunnablePassthrough()|fewShotPromptTemplate|llmWithTools

In [70]:
response=chain.invoke(input={'query':'What is 119 times 8 plus 20 times 23'})

In [71]:
response

AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"y": 8.0, "x": 119.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001', 'safety_ratings': []}, id='run-f68b94ed-bbba-4ab6-8c7e-7875a7e9b9ba-0', tool_calls=[{'name': 'multiply', 'args': {'y': 8.0, 'x': 119.0}, 'id': '1f87f9f2-efa2-44ed-b9bd-ce264dd12eeb', 'type': 'tool_call'}], usage_metadata={'input_tokens': 161, 'output_tokens': 5, 'total_tokens': 166, 'input_token_details': {'cache_read': 0}})

In [72]:
response.tool_calls

[{'name': 'multiply',
  'args': {'y': 8.0, 'x': 119.0},
  'id': '1f87f9f2-efa2-44ed-b9bd-ce264dd12eeb',
  'type': 'tool_call'}]

<h3>Game Changer</h3>

In [131]:
llmWithTools=llmOpenAI.bind_tools(
    tools=tools
)

In [132]:
query='What is 119 times 8 plus 20 times 23'

In [133]:
message=[HumanMessage(content=query,id=1)]

In [134]:
response=llmWithTools.invoke(input=message)
response

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Z4ce1iSqpS28VSSXmOgU2FjJ', 'function': {'arguments': '{"a": 119, "b": 8}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_6jJbadlbJ4j4Kq9BEtYwFFSX', 'function': {'arguments': '{"a": 20, "b": 23}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 98, 'total_tokens': 148, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Be2hUFsDLsyMiuzN43PLpt0WicXyO', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-bccc2ae4-5ce8-4267-acea-6745196f5594-0', tool_calls=[{'name': 'multiply', 'args': {'a': 119, 'b': 8}, 'id': 'call_Z4ce1iSqpS28VSSXmOgU2FjJ', 'type': 'tool_call'}, {'name': 

In [135]:
message.append(response)

In [136]:
message

[HumanMessage(content='What is 119 times 8 plus 20 times 23', additional_kwargs={}, response_metadata={}, id='1'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Z4ce1iSqpS28VSSXmOgU2FjJ', 'function': {'arguments': '{"a": 119, "b": 8}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_6jJbadlbJ4j4Kq9BEtYwFFSX', 'function': {'arguments': '{"a": 20, "b": 23}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 98, 'total_tokens': 148, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Be2hUFsDLsyMiuzN43PLpt0WicXyO', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-bccc2ae4-5ce8-4267-acea-6745196f5594-0', tool_calls=[{'n

In [137]:
# llmWithAlwaysCall.invoke(input=message)  # will throw an error, hence needs to be executed the below code

In [138]:
for toolCall in message[-1].tool_calls:
    selectedTool={"add":add,"multiply":multiply}[toolCall['name'].lower()]
    toolOutput=selectedTool.invoke(input=toolCall['args'])
    message.append(ToolMessage(content=toolOutput,tool_call_id=toolCall['id']))

In [139]:
message

[HumanMessage(content='What is 119 times 8 plus 20 times 23', additional_kwargs={}, response_metadata={}, id='1'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Z4ce1iSqpS28VSSXmOgU2FjJ', 'function': {'arguments': '{"a": 119, "b": 8}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_6jJbadlbJ4j4Kq9BEtYwFFSX', 'function': {'arguments': '{"a": 20, "b": 23}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 98, 'total_tokens': 148, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Be2hUFsDLsyMiuzN43PLpt0WicXyO', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-bccc2ae4-5ce8-4267-acea-6745196f5594-0', tool_calls=[{'n

In [140]:
response=llmWithTools.invoke(input=message)
response

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_T5aIVAKIPBcLctb9dDSJjOgp', 'function': {'arguments': '{"a":952,"b":460}', 'name': 'add'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 164, 'total_tokens': 181, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Be2hi5hPBVxCzCQ5qekzPgLILuCe1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-6eacf92d-c5df-404e-b40b-e9487c8baa7c-0', tool_calls=[{'name': 'add', 'args': {'a': 952, 'b': 460}, 'id': 'call_T5aIVAKIPBcLctb9dDSJjOgp', 'type': 'tool_call'}], usage_metadata={'input_tokens': 164, 'output_tokens': 17, 'total_tokens': 181, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token

In [141]:
message.append(response)

In [142]:
message

[HumanMessage(content='What is 119 times 8 plus 20 times 23', additional_kwargs={}, response_metadata={}, id='1'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Z4ce1iSqpS28VSSXmOgU2FjJ', 'function': {'arguments': '{"a": 119, "b": 8}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_6jJbadlbJ4j4Kq9BEtYwFFSX', 'function': {'arguments': '{"a": 20, "b": 23}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 98, 'total_tokens': 148, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Be2hUFsDLsyMiuzN43PLpt0WicXyO', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-bccc2ae4-5ce8-4267-acea-6745196f5594-0', tool_calls=[{'n

In [143]:
for toolCall in message[-1].tool_calls:
    selectedTool={"add":add,"multiply":multiply}[toolCall['name'].lower()]
    toolOutput=selectedTool.invoke(input=toolCall['args'])
    message.append(ToolMessage(content=toolOutput,tool_call_id=toolCall['id']))

In [144]:
message

[HumanMessage(content='What is 119 times 8 plus 20 times 23', additional_kwargs={}, response_metadata={}, id='1'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Z4ce1iSqpS28VSSXmOgU2FjJ', 'function': {'arguments': '{"a": 119, "b": 8}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_6jJbadlbJ4j4Kq9BEtYwFFSX', 'function': {'arguments': '{"a": 20, "b": 23}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 98, 'total_tokens': 148, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Be2hUFsDLsyMiuzN43PLpt0WicXyO', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-bccc2ae4-5ce8-4267-acea-6745196f5594-0', tool_calls=[{'n

In [145]:
response=llmWithTools.invoke(input=message)
response

AIMessage(content='The result of \\( 119 \\times 8 + 20 \\times 23 \\) is \\( 1412 \\).', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 190, 'total_tokens': 217, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Be2ipslRkGMZ3mqDjNDCUkWuR9nFf', 'finish_reason': 'stop', 'logprobs': None}, id='run-56a99675-9e87-4c0d-a211-b5d22f5a8fb4-0', usage_metadata={'input_tokens': 190, 'output_tokens': 27, 'total_tokens': 217, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [147]:
print(response.content)

The result of \( 119 \times 8 + 20 \times 23 \) is \( 1412 \).
